In [1]:
import logging
import logging.config
import os
import json
import sys



import httplib2

#from googleapiclient.discovery import build
from apiclient import discovery
from apiclient import errors 
from oauth2client import file, client, tools
from oauth2client.file import Storage

APP_SHORT_NAME = 'TeamDriveFolders'

In [2]:
class GDriveError(Exception):
    pass

In [26]:
class googledrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        if  not isinstance(object, discovery.Resource):
            print ('Error: googleapicleint.discovery.Resource object expected')
            print ('{:>5}create a resource object:'.format(''))
            print ('{:>10}credentials = getCredentials(credJSON = "cleint_secret.json")'.format(''))
            print ('{:>10}service = discovery.build("drive", "v3", credentials=credentials)'.format(''))
            print ('{:>10}myDrive = gDrive(service)'.format(''))
            return(None)
        self.service = object
        # https://developers.google.com/drive/v3/web/mime-types
        self.mimeTypes = {'audio': 'application/vnd.google-apps.audio',
                          'docs': 'application/vnd.google-apps.document',
                          'drawing': 'application/vnd.google-apps.drawing',
                          'file': 'application/vnd.google-apps.file',
                          'folder': 'application/vnd.google-apps.folder',
                          'forms': 'application/vnd.google-apps.form',
                          'mymaps': 'application/vnd.google-apps.map',
                          'photos': 'application/vnd.google-apps.photo',
                          'slides': 'application/vnd.google-apps.presentation',
                          'scripts': 'application/vnd.google-apps.script',
                          'sites': 'application/vnd.google-apps.sites',
                          'sheets': 'application/vnd.google-apps.spreadsheet',
                          'video': 'application/vnd.google-apps.video'}
        
        # fields to include in partial responses
        # https://developers.google.com/apis-explorer/#p/drive/v3/drive.files.create
        self.fields = ['id', 'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed', 'kind', 'name']
    
#     types = property()
    
    @property
    def types(self):
        '''
        Display supported mimeTypes
        '''
        print('supported mime types:')
        for key in self.mimeTypes:
            #print('%10s: %s' % (key, self.mimeTypes[key]))
            print('{:8} {val}'.format(key+':', val=self.mimeTypes[key]))
    
#     def quote(self, string):
#         '''
#         add double quotes arounda string
#         '''
#         return('"'+str(string)+'"')
    

    
    def add(self, name = None, mimeType = False, parents = None, 
            fields = 'webViewLink, mimeType, id', sanitize = True):
        '''
        add a file to google drive:
        
        args:
            name (string): human readable name
            mimeType (string): mimeType (see self.mimeTypes for a complete list)
            parents (list): list of parent folders
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        '''

        fieldsExpected = self.fields
        fieldsProcessed = []
        fieldsUnknown = []
        
        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
            
        if len(fieldsUnknown) > 0:
            logging.warn('unrecognized fields: {}'.format(fieldsUnknown))
        
        
        body={}
        if name is None:
            logging.error('expected a folder or file name')
            return(False)
        else:
            body['name'] = name
        
        if mimeType in self.mimeTypes:
            body['mimeType'] = self.mimeTypes[mimeType]
        
        if isinstance(parents, list):
            body['parents'] = parents
        elif parents:
            body['parents'] = [parents]
        
        apiString = 'body={}, fields={}'.format(body, ','.join(fieldsProcessed))
        logging.debug('api call: files().create({})'.format(apiString))
        try:
            result = self.service.files().create(body=body, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        
        
        #body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
# service.files().create(body=body).execute()
        

    
    def Xsearch(self, name = None, trashed = None, mimeType = False, fuzzy = False, date = None, dopperator = '>', 
               parents = None, orderBy = 'createdTime', quiet = True):
        '''
        search for an item by name and other properties in google drive
        
        args:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parents = (string): google drive file id string
            orderBy = (comma separated string): order results assending by keys below - default createdTime:
                        'createdTime', 'folder', 'modifiedByMeTime', 
                        'modifiedTime', 'name', 'quotaBytesUsed', 
                        'recency', 'sharedWithMeTime', 'starred', 
                        'viewedByMeTime'
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
                        
                        
            
        returns:
            list of file dict
        '''
        features = ['name', 'trashed', 'mimeType', 'date', 'parents']
        build = {'name' : 'name {} "{}"'.format(('contains' if fuzzy else '='), name),
                 'trashed' : 'trashed={}'. format(trashed),
                 'mimeType' : 'mimeType="{}"'.format(self.mimeTypes[mimeType] if mimeType in self.mimeTypes else ''),
                 'parents': '"{}" in parents'.format(parents),
                 'date': 'modifiedTime{}"{}"'.format(dopperator, date)}

        
        # provides for setting trashed to True/False if the input is not None
        if not isinstance(trashed, type(None)):
            # set to true as the variable is now in use, but it's value has been set above
            trashed = True
        
        qList = []

        # evaluate feature options; if they are != None/False, use them in building query
        for each in features:
            if eval(each):
                qList.append(build[each])
                
        if not quiet:
            print(' and '.join(qList))
        
        apiString = 'q={}, orderBy={})'.format(' and '.join(qList), orderBy)
        logging.debug('apicall: files().list({})'.format(apiString))
        try:
            # build a query with "and" statements

            result = self.service.files().list(q=' and '.join(qList), orderBy=orderBy).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        
    def search(self, name = None, trashed = None, mimeType = False, fuzzy = False, date = None, dopperator = '>', 
               parents = None, orderBy = 'createdTime', teamdrive = None, quiet = True):
        '''
        search for an item by name and other properties in google drive
        
        args:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parents = (string): google drive file id string
            orderBy = (comma separated string): order results assending by keys below - default createdTime:
                        'createdTime', 'folder', 'modifiedByMeTime', 
                        'modifiedTime', 'name', 'quotaBytesUsed', 
                        'recency', 'sharedWithMeTime', 'starred', 
                        'viewedByMeTime'
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            teamdrive (string): Team Drive ID string - when included only the specified Team Drive is searched
            quiet (bool): false prints all the results
                        
                        
            
        returns:
            list of file dict
        '''
        features = ['name', 'trashed', 'mimeType', 'date', 'parents']
        build = {'name' : 'name {} "{}"'.format(('contains' if fuzzy else '='), name),
                 'trashed' : 'trashed={}'. format(trashed),
                 'mimeType' : 'mimeType="{}"'.format(self.mimeTypes[mimeType] if mimeType in self.mimeTypes else ''),
                 'parents': '"{}" in parents'.format(parents),
                 'date': 'modifiedTime{}"{}"'.format(dopperator, date)}


    
            
        # provides for setting trashed to True/False if the input is not None
        if not isinstance(trashed, type(None)):
            # set to true as the variable is now in use, but it's value has been set above
            trashed = True
        
        qList = []

        # evaluate feature options; if they are != None/False, use them in building query
        for each in features:
            if eval(each):
                qList.append(build[each])
                
        if not quiet:
            print(' and '.join(qList))
        
        apiString = 'q={}, orderBy={})'.format(' and '.join(qList), orderBy)
        logging.debug('apicall: files().list({})'.format(apiString))
        try:
            # build a query with "and" statements

            if teamdrive:
                result = self.service.files().list(q=' and '.join(qList), 
                                                   orderBy=orderBy, 
                                                   corpora='teamDrive',
                                                   includeTeamDriveItems='true',
                                                   teamDriveId=teamdrive, 
                                                   supportsTeamDrives='true').execute()
            else:
                result = self.service.files().list(q=' and '.join(qList), orderBy=orderBy).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)

    def ls(self, *args, **kwargs):
        '''
        List files in google drive using any of the following properties:
            
        accepts:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parent = (string): google drive file id string    
        '''
        try:
            result = self.search(*args, **kwargs)
            for eachFile in result.get('files', []):
                print('name: {f[name]}, ID:{f[id]}, mimeType:{f[mimeType]}'.format(f=eachFile))
            return(result)
        except GDriveError as e:
            raise GDriveError(e)
            
    
    
    def getprops(self, fileId = None, fields = 'parents, mimeType, webViewLink', sanitize=True):
        '''
        get a file or folder's properties based on google drive fileId
        
        for a more complete list: https://developers.google.com/drive/v3/web/migration
        
        args:
            fileId (string): google drive file ID
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        returns:
            list of dictionary - google drive file properties
            
        raises GDriveError
        '''
        fieldsExpected = self.fields
        
        fieldsProcessed = []
        fieldsUnknown = []

        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
        if len(fieldsUnknown) > 0:
            print ('unrecognized fields: {}'.format(fieldsUnknown))
        
        apiString = 'fileId={}, fields={}'.format(fileId, ','.join(fieldsProcessed))
        logging.debug('files().get({})'.format(apiString))
        try:
            result = self.service.files().get(fileId=fileId, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        

    def parents(self, fileId):
        """get a file's parents.

        Args:
            fileId: ID of the file to print parents for.
        
        raises GDriveError
        """
        apiString = 'fileId={}, fields="parents"'.format(fileId)
        logging.debug('api call: {}'.format(apiString))
        try:
            parents = self.service.files().get(fileId=fileId, fields='parents').execute()
            return(parents)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
    
    def rm(self):
        pass
    
    def listTeamDrives(self):
        '''
        List first page of team drives available to the user 
            - this method ignores the continuation token (I can't figure it out!)
            raises GDriveError
            
            returns: 
                dictonary of first page of TeamDrives
        '''
        try:
            result = self.service.teamdrives().list().execute()
            return(result['teamDrives'])
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        

In [4]:
def getCredentials(config_path = os.path.expanduser('~/.config/'+APP_SHORT_NAME), 
                   client_secret = './client_secrets.json'):
    scopes = 'https://www.googleapis.com/auth/drive' # this is a bit expansive, consider a slimmer set
    
    # update this - this is not valid; need to pass in client_secret from the main module
    #client_secret = './client_secret_'+APP_SHORT_NAME+'.json'
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(config_path, 'credentials')
    credential_file = os.path.join(credential_dir, APP_SHORT_NAME+'_credentials.json')
    flags = tools.argparser.parse_args([])

    if not os.path.exists(client_secret):
        logging.critical('fatal error - missing client secret file: {}'.format(client_secret))
        logging.critical('please obtain a client secret file and place it in the same dirctory as this script')
        logging.critical('filename: {}'.format(client_secret))
        logging.critical('instructions: https://developers.google.com/drive/v3/web/quickstart/python')
        sys.exit(1)
        
    logging.debug('checking for credential store directory: {}'.format(credential_dir))
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            logging.critical(e)
    
    store = Storage(credential_file)
    creds = store.get()
    
    
    if not creds or creds.invalid:
        logging.debug('credential store not found or is invalid; refreshing')
        flow = client.flow_from_clientsecrets(client_secret, scopes)
        logging.debug('preparing to set store')
        creds = tools.run_flow(flow, store, flags)
    else:
        logging.debug('credential store accepted')
        
    
    return(creds)

In [5]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [27]:
def main(): 
    logger = logging.getLogger(__name__)
    setup_logging(default_level=logging.DEBUG)
    
    
    
    logging.info('checking google drive credentials')
    try:
        credentials = getCredentials()
    except SystemExit:
        logging.critical('You have chosen to deny access to google drive.')
        logging.critical('This program cannot continue without access to google drive.')
        
    http = credentials.authorize(httplib2.Http())
    
    logging.debug('building api discovery service')
    try:
        service = discovery.build('drive', 'v3', http=http, cache_discovery=False)
    except Exception as e:
        logging.critical('Error communicating with Google: {}'.format(e))
        logging.critical('exiting')
        return(False)

    logging.debug('preparing google drive object')
    myDrive = googledrive(service)
    return(myDrive)
    
    

    
#if __name__ == '__main__':
#    main() 
foo=main()

2018-08-13 01:23:08,622: [INFO: root.main] checking google drive credentials
2018-08-13 01:23:08,623: [DEBUG: root.getCredentials] checking for credential store directory: /Users/aaronciuffo/.config/TeamDriveFolders/credentials
2018-08-13 01:23:08,624: [DEBUG: root.getCredentials] credential store accepted
2018-08-13 01:23:08,625: [DEBUG: root.main] building api discovery service
2018-08-13 01:23:08,626: [INFO: googleapiclient.discovery._retrieve_discovery_doc] URL being requested: GET https://www.googleapis.com/discovery/v1/apis/drive/v3/rest
2018-08-13 01:23:08,695: [DEBUG: root.main] preparing google drive object


In [ ]:
bar = foo.service.files().list(q="name contains 'Fey'", corpora='teamDrive',includeTeamDriveItems='true',teamDriveId='0ACLfU8KeD_BHUk9PVA',supportsTeamDrives='true').execute()


In [30]:
bar = foo.search(name='Fey', teamdrive='0ACLfU8KeD_BHUk9PVA', fuzzy=True)
bar

2018-08-13 01:24:01,468: [DEBUG: root.search] apicall: files().list(q=name contains "Fey", orderBy=createdTime))
2018-08-13 01:24:01,473: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&corpora=teamDrive&supportsTeamDrives=true&q=name+contains+%22Fey%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json


{u'files': [{u'id': u'0B9WTleJ1MzaYY0I5ZFY3blNHZjQ',
   u'kind': u'drive#file',
   u'mimeType': u'application/vnd.google-apps.folder',
   u'name': u'Feynman, Richard - 554264',
   u'teamDriveId': u'0ACLfU8KeD_BHUk9PVA'}],
 u'incompleteSearch': False,
 u'kind': u'drive#fileList'}

In [ ]:
for each in bar:
    print each